See [BenchmarkDotNet Getting Started](https://benchmarkdotnet.org/articles/guides/getting-started.html) for original documentation.

### 1. Create a new console app

Add a project `BenchmarkDotNet.Sample` so we can add a simple benchmark and check the results.

In [ ]:
dotnet new console --framework net6.0 -o BenchmarkDotNet.Sample

### 2. Install BenchmarkDotNet

In [ ]:
# Add nuget.org as a source in case it is not already present
dotnet nuget add source https://api.nuget.org/v3/index.json -n nuget.org

In [ ]:
# Install BenchmarkDotNet nuget package
dotnet add ./BenchmarkDotNet.Sample/BenchmarkDotNet.Sample.csproj package BenchmarkDotNet

### 3. Add a benchmark

Add a class with methods decorated by `[Benchmark]`. See the example below in `Program.cs`.

```c#
using System;
using System.Security.Cryptography;
using BenchmarkDotNet.Attributes;
using BenchmarkDotNet.Running;

namespace MyBenchmarks
{
    public class Md5VsSha256
    {
        private const int N = 100;
        private readonly byte[] data;

        private readonly SHA256 sha256 = SHA256.Create();
        private readonly MD5 md5 = MD5.Create();

        public Md5VsSha256()
        {
            data = new byte[N];
            new Random(42).NextBytes(data);
        }

        [Benchmark]
        public byte[] Sha256() => sha256.ComputeHash(data);

        [Benchmark]
        public byte[] Md5() => md5.ComputeHash(data);
    }

    public class Program
    {
        public static void Main(string[] args)
        {
            var summary = BenchmarkRunner.Run<Md5VsSha256>();
        }
    }
}
```

The `BenchmarkRunner.Run<Md5VsSha256>()` call runs your benchmarks and print results to console output.


### 4. dotnet run and view results

In [ ]:
dotnet run -c Release --project ./BenchmarkDotNet.Sample/BenchmarkDotNet.Sample.csproj

Benchmarks will only run in a **Release** build. A `BenchmarkDotNet.Artifacts` directory will be created for the results after `dotnet run`. The summary like below will also be printed out.

```bash
// * Summary *

BenchmarkDotNet=v0.13.1, OS=Windows 10.0.25179
AMD Ryzen 7 6800HS with Radeon Graphics, 1 CPU, 16 logical and 8 physical cores
.NET SDK=6.0.400
  [Host]     : .NET 6.0.8 (6.0.822.36306), X64 RyuJIT
  DefaultJob : .NET 6.0.8 (6.0.822.36306), X64 RyuJIT


| Method |      Mean |     Error |    StdDev |
|------- |----------:|----------:|----------:|
| Sha256 |  6.536 us | 0.0048 us | 0.0045 us |
|    Md5 | 21.449 us | 0.0166 us | 0.0147 us |

// * Hints *
Outliers
  Md5VsSha256.Md5: Default -> 1 outlier  was  removed (21.60 us)

// * Legends *
  Mean   : Arithmetic mean of all measurements
  Error  : Half of 99.9% confidence interval
  StdDev : Standard deviation of all measurements
  1 us   : 1 Microsecond (0.000001 sec)

// ***** BenchmarkRunner: End *****
// ** Remained 0 benchmark(s) to run **
Run time: 00:00:44 (44.08 sec), executed benchmarks: 2

Global total time: 00:00:48 (48.33 sec), executed benchmarks: 2
// * Artifacts cleanup *
```